In [1]:
# Imports and setup
from pydrake.all import *
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.meshcat_utils import AddMeshcatTriad
from pathlib import Path
import numpy as np
import sys

sys.path.append(str(Path.cwd()))
from throw_helpers import interpolate_poses_linear, create_q_knots

In [2]:
# Start meshcat
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [4]:
# Scene configuration
TABLE_SDF_PATH = f"{Path.cwd()}/assets/table.sdf"

# Ball position: in front of robot, on table surface
BALL_POS = [0, -0.5, 0.025]  # x=side, y=forward, z=up

scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: file://{TABLE_SDF_PATH}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: ball
    file: package://drake_models/manipulation_station/sphere.sdf
    default_free_body_pose:
        base_link:
            translation: {BALL_POS}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

In [5]:
# Build and visualize initial scene
meshcat.Delete()

builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")

# Get initial poses
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)

X_WG_initial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
ball_body = plant.GetBodyByName("base_link", plant.GetModelInstanceByName("ball"))
X_WBall = plant.EvalBodyPoseInWorld(temp_plant_context, ball_body)

print(f"Gripper initial pose: {X_WG_initial}")
print(f"Ball pose: {X_WBall}")

# Add frame triad attached to gripper body
AddFrameTriadIllustration(
    scene_graph=scene_graph,
    body=plant.GetBodyByName("body"),
    length=0.18,
    radius=0.005,
)

# Build diagram and publish to meshcat (no simulation yet)
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

Gripper initial pose: RigidTransform(
  R=RotationMatrix([
    [0.999999682931835, 0.00019052063137842197, -0.0007731999219133521],
    [0.0007963267107334455, -0.2392492533556365, 0.9709578572896667],
    [1.8684985011115333e-16, -0.9709581651495918, -0.23924932921398262],
  ]),
  p=[0.0003707832187589982, -0.46561680802324634, 0.6793215789060889],
)
Ball pose: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, -0.5, 0.025],
)


In [6]:
# Pick trajectory parameters
GRIPPER_OPEN = 0.107
GRIPPER_CLOSED = 0.0
GRASP_HEIGHT = 0.12  # Height above ball center to grasp
APPROACH_HEIGHT = 0.1  # Height above grasp pose for approach

def design_grasp_pose(X_WO: RigidTransform) -> RigidTransform:
    """Grasp pose above object with gripper pointing down."""
    return X_WO @ RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0, 0, GRASP_HEIGHT])

def make_pick_trajectory(
    X_WG_start,
    X_WBall,
    t_approach=1.0,
    t_descend=0.5,
    t_grasp=0.5,
    t_lift=0.5,
):
    """
    Generate pick trajectory: approach -> descend -> grasp -> lift.
    Returns: (q_traj, gripper_traj, total_time, X_WG_end)
    """
    X_WG_grasp = design_grasp_pose(X_WBall)
    X_WG_approach = RigidTransform(p=[0, 0, APPROACH_HEIGHT]) @ X_WG_grasp
    
    # Keyframe times
    t0, t1, t2, t3 = 0, t_approach, t_approach + t_descend, t_approach + t_descend + t_grasp
    t4 = t3 + t_lift
    
    # Pose trajectory: start -> approach -> grasp -> hold -> lift back to approach
    pose_traj = PiecewisePose.MakeLinear(
        [t0, t1, t2, t3, t4],
        [X_WG_start, X_WG_approach, X_WG_grasp, X_WG_grasp, X_WG_approach]
    )
    
    # Gripper trajectory: open -> open -> close -> close -> close
    gripper_traj = PiecewisePolynomial.FirstOrderHold(
        [t0, t1, t2, t3, t4],
        np.array([[GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_CLOSED, GRIPPER_CLOSED]])
    )
    
    # Convert to joint trajectory via IK
    num_samples = 50
    t_samples = np.linspace(t0, t4, num_samples)
    poses = [pose_traj.GetPose(t) for t in t_samples]
    q_knots = create_q_knots(plant, poses)
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_samples[:len(q_knots)], np.array(q_knots).T)
    
    return q_traj, gripper_traj, t4, X_WG_approach

In [7]:
# Generate pick trajectory
q_traj, gripper_traj, total_time, X_WG_postpick = make_pick_trajectory(X_WG_initial, X_WBall)
print(f"Pick trajectory: {total_time:.1f}s, {q_traj.get_number_of_segments()} segments")

Pick trajectory: 2.5s, 49 segments


In [8]:
# Step 1: Define throw joint angles
from throw_helpers import joint_angles_to_pose

# Target position
P_TARGET = np.array([0, 1.0, 0.0])

# Compute throw heading (direction to target)
throw_heading = np.arctan2(P_TARGET[1], P_TARGET[0])
print(f"Throw heading: {np.degrees(throw_heading):.1f} deg")

# Joint angle 1 points opposite to throw direction (arm winds up behind)
ja1 = throw_heading - np.pi

# Prethrow: wound up position (joints 4 and 6 bent)
# Throwend: extended position (joints 4 and 6 straightened)
PRETHROW_JA = np.array([ja1, 0, 0, 1.9, 0, -1.9, np.pi])
THROWEND_JA = np.array([ja1, 0, 0, 0.4, 0, -0.4, np.pi])

print(f"PRETHROW_JA: {PRETHROW_JA}")
print(f"THROWEND_JA: {THROWEND_JA}")

Throw heading: 90.0 deg
PRETHROW_JA: [-1.57079633  0.          0.          1.9         0.         -1.9
  3.14159265]
THROWEND_JA: [-1.57079633  0.          0.          0.4         0.         -0.4
  3.14159265]


In [9]:
# Step 2: Verify postpick, prethrow, throwend positions
# Build a trajectory that moves through all 3 positions with pauses
import time
from throw_helpers import joint_angles_to_pose

q_postpick = q_traj.value(q_traj.end_time()).flatten()

# Define the 3 key positions
positions = {
    "postpick": q_postpick,
    "prethrow": PRETHROW_JA,
    "throwend": THROWEND_JA,
}

# Build trajectory: hold at each position for 2 seconds
t_hold = 2.0
t_lst = [0.0]
q_lst = [q_postpick]
for i, (name, q) in enumerate(positions.items()):
    t_lst.append(t_lst[-1] + 0.5)  # transition
    q_lst.append(q)
    t_lst.append(t_lst[-1] + t_hold)  # hold
    q_lst.append(q)

q_test_traj = PiecewisePolynomial.FirstOrderHold(t_lst, np.array(q_lst).T)
g_test_traj = PiecewisePolynomial.FirstOrderHold([0, t_lst[-1]], np.array([[GRIPPER_CLOSED, GRIPPER_CLOSED]]))

In [10]:
# Build diagram
meshcat.Delete()
builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
test_plant = station.GetSubsystemByName("plant")

# Add triads for each position using FK
for name, q in positions.items():
    T = joint_angles_to_pose(test_plant, q)
    AddMeshcatTriad(meshcat, name, length=0.15, radius=0.006, X_PT=T)
    print(f"{name}: {T.translation()}")

# Add target triad
AddMeshcatTriad(meshcat, "target", length=0.2, radius=0.01, X_PT=RigidTransform(p=P_TARGET))

# Connect trajectory sources
q_source = builder.AddSystem(TrajectorySource(q_test_traj))
g_source = builder.AddSystem(TrajectorySource(g_test_traj))
builder.Connect(q_source.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(g_source.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.AdvanceTo(q_test_traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

print("Done! Arm should have visited: postpick -> prethrow -> throwend")

postpick: [ 1.23637274e-15 -5.00000000e-01  2.45000000e-01]
prethrow: [2.17314828e-16 2.73725115e-01 4.75985823e-01]
throwend: [2.10796307e-17 2.77881841e-01 1.22675236e+00]
Done! Arm should have visited: postpick -> prethrow -> throwend


In [11]:
# Step 3: Compute planned launch angle at release point

RELEASE_FRAC = 0.5  # Release halfway through throw motion
GRIPPER_TO_OBJECT_DIST = 0.12  # Distance from gripper frame to ball COM
THROW_DURATION = 0.4  # Time for throw motion (prethrow -> throwend)

# Release joint angles and velocities
release_ja = PRETHROW_JA + RELEASE_FRAC * (THROWEND_JA - PRETHROW_JA)
qdot_throw = (THROWEND_JA - PRETHROW_JA) / THROW_DURATION

# Use existing test_plant from Step 2 cell
test_plant_context = test_plant.CreateDefaultContext()
iiwa = test_plant.GetModelInstanceByName("iiwa")
test_plant.SetPositions(test_plant_context, iiwa, release_ja)

# Compute Jacobian at release
gripper_body = test_plant.GetBodyByName("body")
J_G = test_plant.CalcJacobianTranslationalVelocity(
    test_plant_context,
    JacobianWrtVariable.kQDot,
    gripper_body.body_frame(),
    [0, GRIPPER_TO_OBJECT_DIST, 0],
    test_plant.world_frame(),
    test_plant.world_frame()
)
iiwa_start = test_plant.GetJointByName("iiwa_joint_1").position_start()
J_iiwa = J_G[:, iiwa_start:iiwa_start+7]

# Ball velocity at release
v_release = J_iiwa @ qdot_throw
launch_vx = np.linalg.norm(v_release[:2])
launch_vy = v_release[2]
planned_launch_angle = np.arctan2(launch_vy, launch_vx)

# Release position
T_release = joint_angles_to_pose(test_plant, release_ja)
p_release = T_release.translation() + T_release.rotation() @ np.array([0, GRIPPER_TO_OBJECT_DIST, 0])

print(f"Release position: {p_release}")
print(f"Release velocity: {v_release} (speed: {np.linalg.norm(v_release):.2f} m/s)")
print(f"Planned launch angle: {np.degrees(planned_launch_angle):.1f} deg")

Release position: [1.61575508e-16 5.81276558e-01 7.09637548e-01]
Release velocity: [-3.11911213e-16  8.39683634e-01  2.99213971e+00] (speed: 3.11 m/s)
Planned launch angle: 74.3 deg


In [19]:
# Step 4: Build throw trajectory (prethrow -> throwend)
# Using linear joint interpolation matching the reference implementation

def interpolate_joint_angles(ja_start, ja_end, duration, num_samples):
    """
    Constant joint velocity interpolation between two configurations.
    Returns: (t_samples, q_samples) where q_samples is list of joint angle arrays
    """
    t_samples = np.linspace(0, duration, num_samples, endpoint=True)
    q_samples = []
    for t in t_samples:
        frac = t / duration
        q_samples.append(ja_start + frac * (ja_end - ja_start))
    return t_samples, q_samples

# Generate throw trajectory samples
NUM_THROW_SAMPLES = 30
t_throw_samples, q_throw_samples = interpolate_joint_angles(
    PRETHROW_JA, THROWEND_JA, THROW_DURATION, NUM_THROW_SAMPLES
)

# Create smooth trajectory using CubicShapePreserving (matches reference)
q_throw_traj = PiecewisePolynomial.CubicShapePreserving(
    t_throw_samples, 
    np.array(q_throw_samples).T
)

print(f"Throw trajectory: {THROW_DURATION:.2f}s, {q_throw_traj.get_number_of_segments()} segments")
print(f"Time range: [{q_throw_traj.start_time():.3f}, {q_throw_traj.end_time():.3f}]")

Throw trajectory: 0.40s, 29 segments
Time range: [0.000, 0.400]


In [20]:
# Step 5: Visualize triads at sampled points along throw trajectory
meshcat.Delete()

# Build station for FK calculations
builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
throw_plant = station.GetSubsystemByName("plant")

# Sample trajectory at regular intervals and add triads
VIZ_SAMPLES = 10
viz_times = np.linspace(0, THROW_DURATION, VIZ_SAMPLES)  # Fixed: use trajectory time range [0, THROW_DURATION]
for i, t in enumerate(viz_times):
    q = q_throw_traj.value(t).flatten()
    T = joint_angles_to_pose(throw_plant, q)
    frac = t / THROW_DURATION
    AddMeshcatTriad(meshcat, f"throw_sample_{i}", length=0.08, radius=0.003, X_PT=T)
    print(f"t={t:.3f}s (frac={frac:.2f}): pos={T.translation()}")

# Add key poses
AddMeshcatTriad(meshcat, "prethrow", length=0.15, radius=0.006, X_PT=joint_angles_to_pose(throw_plant, PRETHROW_JA))
AddMeshcatTriad(meshcat, "throwend", length=0.15, radius=0.006, X_PT=joint_angles_to_pose(throw_plant, THROWEND_JA))
AddMeshcatTriad(meshcat, "release", length=0.12, radius=0.005, X_PT=joint_angles_to_pose(throw_plant, release_ja))
AddMeshcatTriad(meshcat, "target", length=0.2, radius=0.01, X_PT=RigidTransform(p=P_TARGET))

print(f"\nVisualized {VIZ_SAMPLES} triads along throw trajectory")

t=0.000s (frac=0.00): pos=[2.17314828e-16 2.73725115e-01 4.75985823e-01]
t=0.044s (frac=0.11): pos=[2.06926884e-16 3.39780450e-01 5.13781575e-01]
t=0.089s (frac=0.22): pos=[1.90733696e-16 4.00904792e-01 5.71155610e-01]
t=0.133s (frac=0.33): pos=[1.69229600e-16 4.50802647e-01 6.47252965e-01]
t=0.178s (frac=0.44): pos=[1.43551712e-16 4.83502417e-01 7.39153731e-01]
t=0.222s (frac=0.56): pos=[1.15386110e-16 4.94027052e-01 8.42043257e-01]
t=0.267s (frac=0.67): pos=[8.68171311e-17 4.78980121e-01 9.49595225e-01]
t=0.311s (frac=0.78): pos=[6.01211538e-17 4.36981732e-01 1.05452951e+00]
t=0.356s (frac=0.89): pos=[3.75466415e-17 3.68904583e-01 1.14928727e+00]
t=0.400s (frac=1.00): pos=[2.10795396e-17 2.77881841e-01 1.22675236e+00]

Visualized 10 triads along throw trajectory


In [21]:
# Step 6: Simulate throw motion (prethrow -> throwend)
# Gripper stays closed throughout this demo

# Prepend "go to prethrow" segment (1 second hold at prethrow to let arm settle)
T_SETTLE = 1.0
t_full = np.concatenate([[0], t_throw_samples + T_SETTLE])
q_full = [PRETHROW_JA] + q_throw_samples  # Start at prethrow, hold, then throw
q_full_traj = PiecewisePolynomial.CubicShapePreserving(t_full, np.array(q_full).T)

g_throw_traj = PiecewisePolynomial.FirstOrderHold(
    [0, q_full_traj.end_time()], 
    np.array([[GRIPPER_CLOSED, GRIPPER_CLOSED]])
)

# Connect trajectory sources
q_source = builder.AddSystem(TrajectorySource(q_full_traj))
g_source = builder.AddSystem(TrajectorySource(g_throw_traj))
builder.Connect(q_source.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(g_source.get_output_port(), station.GetInputPort("wsg.position"))

# Build and run
diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(0.5)  # Slow motion to see the throw

meshcat.StartRecording()
simulator.AdvanceTo(q_full_traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

print(f"Done! Arm held at prethrow for {T_SETTLE}s, then threw in {THROW_DURATION:.2f}s")

Done! Arm held at prethrow for 1.0s, then threw in 0.40s
